In [1]:
import yfinance as yf
import backtrader as bt
import matplotlib.pyplot as plt
print("✅ Imports OK")

✅ Imports OK


In [2]:
import yfinance as yf
import backtrader as bt
import matplotlib.pyplot 
import pandas as pd

def get_data(symbols, start="2023-01-01", end="2024-01-01"):
    """
    Fetch historical OHLCV data for one or multiple tickers.
    Accepts a string ('AAPL') or list of strings (['AAPL', 'MSFT', 'GOOG']).
    Returns a dictionary {symbol: DataFrame}.
    """
    if isinstance(symbols, str):
        symbols = [symbols]

    data_dict = {}
    for symbol in symbols:
        df = yf.download(symbol, start=start, end=end, progress=False)
        if not df.empty:
            df.dropna(inplace=True)
            data_dict[symbol] = df
        else:
            print(f"⚠️ No data for {symbol}")
    return data_dict


In [3]:
stocks = get_data(["AAPL", "MSFT", "GOOG"])
for k,v in stocks.items():
    print(k, v.shape)


C:\Users\olusa\AppData\Local\Temp\ipykernel_46708\495773760.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start, end=end, progress=False)
C:\Users\olusa\AppData\Local\Temp\ipykernel_46708\495773760.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start, end=end, progress=False)
C:\Users\olusa\AppData\Local\Temp\ipykernel_46708\495773760.py:17: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(symbol, start=start, end=end, progress=False)


AAPL (250, 5)
MSFT (250, 5)
GOOG (250, 5)


In [4]:

import backtrader as bt


In [5]:
class SmaCross(bt.Strategy):
    """
    Simple Moving Average crossover strategy.
    Buys when fast SMA crosses above slow SMA; sells when it crosses below.
    """
    params = dict(pfast=10, pslow=30)

    def __init__(self):
        sma1 = bt.ind.SMA(period=self.p.pfast)
        sma2 = bt.ind.SMA(period=self.p.pslow)
        self.crossover = bt.ind.CrossOver(sma1, sma2)

    def next(self):
        if not self.position and self.crossover > 0:
            self.buy()
        elif self.position and self.crossover < 0:
            self.sell()


In [6]:
SmaCross


__main__.SmaCross

In [7]:
stocks["MSFT"].columns.tolist(), type(stocks["MSFT"].columns)



([('Close', 'MSFT'),
  ('High', 'MSFT'),
  ('Low', 'MSFT'),
  ('Open', 'MSFT'),
  ('Volume', 'MSFT')],
 pandas.core.indexes.multi.MultiIndex)

In [8]:
%%writefile litmus_app.py
import streamlit as st
import yfinance as yf
import backtrader as bt
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import datetime

# ---------- Helpers ----------
def clean_data(data):
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)
    data.columns = [c.lower().strip() for c in data.columns]
    data.dropna(inplace=True)
    return data

# ---------- Base Strategy ----------
class BaseStrategy(bt.Strategy):
    def __init__(self):
        self.signals = []
        self.trades = []

    def log_signal(self, action):
        dt, price = self.data.datetime.date(0), self.data.close[0]
        self.signals.append((dt, action, price))

    def notify_trade(self, trade):
        if trade.isclosed:
            self.trades.append(trade.pnl)

# ---------- 17 Strategies ----------
class SmaCross(BaseStrategy):
    params = dict(pfast=10, pslow=30)
    def __init__(self):
        super().__init__()
        sma1, sma2 = bt.ind.SMA(period=self.p.pfast), bt.ind.SMA(period=self.p.pslow)
        self.cross = bt.ind.CrossOver(sma1, sma2)
    def next(self):
        if not self.position and self.cross > 0:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.cross < 0:
            self.sell(); self.log_signal("SELL")

class Rsi(BaseStrategy):
    params = dict(period=14, low=30, high=70)
    def __init__(self):
        super().__init__()
        self.rsi = bt.ind.RSI(period=self.p.period)
    def next(self):
        if not self.position and self.rsi < self.p.low:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.rsi > self.p.high:
            self.sell(); self.log_signal("SELL")

class Bollinger(BaseStrategy):
    params = dict(period=20, dev=2)
    def __init__(self):
        super().__init__()
        self.bb = bt.ind.BollingerBands(period=self.p.period, devfactor=self.p.dev)
    def next(self):
        if not self.position and self.data.close < self.bb.bot:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.data.close > self.bb.top:
            self.sell(); self.log_signal("SELL")

class Momentum(BaseStrategy):
    params = dict(period=20)
    def __init__(self): super().__init__()
    def next(self):
        if len(self.data) <= self.p.period: return
        if not self.position and self.data.close[0] > self.data.close[-self.p.period]:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.data.close[0] < self.data.close[-self.p.period]:
            self.sell(); self.log_signal("SELL")

class MACD(BaseStrategy):
    params = dict(fast=12, slow=26, sig=9)
    def __init__(self):
        super().__init__()
        m = bt.ind.MACD(period_me1=self.p.fast, period_me2=self.p.slow, period_signal=self.p.sig)
        self.cross = bt.ind.CrossOver(m.macd, m.signal)
    def next(self):
        if not self.position and self.cross > 0:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.cross < 0:
            self.sell(); self.log_signal("SELL")

class BuyHold(BaseStrategy):
    def __init__(self): super().__init__()
    def next(self):
        if not self.position:
            self.buy(); self.log_signal("BUY")

class EMA(BaseStrategy):
    params = dict(fast=12, slow=26)
    def __init__(self):
        super().__init__()
        f, s = bt.ind.EMA(period=self.p.fast), bt.ind.EMA(period=self.p.slow)
        self.cross = bt.ind.CrossOver(f, s)
    def next(self):
        if not self.position and self.cross > 0:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.cross < 0:
            self.sell(); self.log_signal("SELL")

class DualSMA(BaseStrategy):
    params = dict(short=50, long=200)
    def __init__(self):
        super().__init__()
        s, l = bt.ind.SMA(period=self.p.short), bt.ind.SMA(period=self.p.long)
        self.cross = bt.ind.CrossOver(s, l)
    def next(self):
        if not self.position and self.cross > 0:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.cross < 0:
            self.sell(); self.log_signal("SELL")

class Donchian(BaseStrategy):
    params = dict(period=20)
    def __init__(self):
        super().__init__()
        self.h = bt.ind.Highest(self.data.high, period=self.p.period)
        self.l = bt.ind.Lowest(self.data.low, period=self.p.period)
    def next(self):
        if not self.position and self.data.close[0] > self.h[-1]:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.data.close[0] < self.l[-1]:
            self.sell(); self.log_signal("SELL")

class Stoch(BaseStrategy):
    params = dict(period=14)
    def __init__(self):
        super().__init__()
        self.st = bt.ind.Stochastic(period=self.p.period)
        self.cross = bt.ind.CrossOver(self.st.percK, self.st.percD)
    def next(self):
        if not self.position and self.cross > 0 and self.st.percK < 20:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.cross < 0 and self.st.percK > 80:
            self.sell(); self.log_signal("SELL")

class ADX(BaseStrategy):
    params = dict(period=14, thr=25)
    def __init__(self):
        super().__init__()
        self.a = bt.ind.ADX(period=self.p.period)
    def next(self):
        if not self.position and self.a > self.p.thr:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.a < self.p.thr:
            self.sell(); self.log_signal("SELL")

class VWAP(BaseStrategy):
    def __init__(self):
        super().__init__()
        self.v = bt.ind.VWAP()
    def next(self):
        if not self.position and self.data.close[0] > self.v[0]:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.data.close[0] < self.v[0]:
            self.sell(); self.log_signal("SELL")

class ATRStop(BaseStrategy):
    params = dict(period=14, m=2)
    def __init__(self):
        super().__init__()
        self.a = bt.ind.ATR(period=self.p.period)
    def next(self):
        if not self.position:
            self.buy(); self.log_signal("BUY"); self.e = self.data.close[0]
        elif self.position and self.data.close[0] < self.e - self.p.m * self.a[0]:
            self.sell(); self.log_signal("SELL")

class Seasonal(BaseStrategy):
    params = dict(months=[11, 12, 1, 2, 3, 4])
    def __init__(self): super().__init__()
    def next(self):
        m = self.data.datetime.date(0).month
        if not self.position and m in self.p.months:
            self.buy(); self.log_signal("BUY")
        elif self.position and m not in self.p.months:
            self.sell(); self.log_signal("SELL")

class RiskParity(BaseStrategy):
    def __init__(self):
        super().__init__()
        self.a = bt.ind.ATR(period=14)
    def next(self):
        if not self.position:
            self.buy(size=(1 / (self.a[0] + 1e-6)) / 100)
            self.log_signal("BUY")

class Factor(BaseStrategy):
    def __init__(self): super().__init__()
    def next(self):
        s, r = bt.ind.SMA(period=50), bt.ind.RSI(period=14)
        if not self.position and self.data.close > s and r < 70:
            self.buy(); self.log_signal("BUY")
        elif self.position and r > 70:
            self.sell(); self.log_signal("SELL")

class Breakout(BaseStrategy):
    params = dict(look=20)
    def __init__(self):
        super().__init__()
        self.h = bt.ind.Highest(self.data.high, period=self.p.look)
        self.l = bt.ind.Lowest(self.data.low, period=self.p.look)
    def next(self):
        if not self.position and self.data.close[0] > self.h[-1]:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.data.close[0] < self.l[-1]:
            self.sell(); self.log_signal("SELL")

STRATS = {
    "SMA Crossover": SmaCross, "RSI": Rsi, "Bollinger": Bollinger, "Momentum": Momentum, "MACD": MACD,
    "Buy & Hold": BuyHold, "EMA": EMA, "Dual SMA": DualSMA, "Donchian": Donchian, "Stoch": Stoch,
    "ADX": ADX, "VWAP": VWAP, "ATR Stop": ATRStop, "Seasonal": Seasonal, "Risk Parity": RiskParity,
    "Factor": Factor, "Breakout": Breakout
}

# ---------- Backtest ----------
def run_bt(data, cls, cash=10000):
    data = clean_data(data.copy())
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(cash)
    cerebro.adddata(bt.feeds.PandasData(dataname=data))
    cerebro.addstrategy(cls)
    results = cerebro.run()
    strat = results[0]
    sigs = getattr(strat, "signals", [])
    trades = getattr(strat, "trades", [])
    start, end = cash, cerebro.broker.getvalue()
    ret = round((end / start - 1) * 100, 2)
    wr = round(100 * len([t for t in trades if t > 0]) / len(trades), 2) if trades else 0
    dd = round(abs(min(trades)) / cash * 100, 2) if trades else 0
    return ret, end, sigs, wr, dd

# ---------- Streamlit ----------
st.set_page_config(page_title="Litmus Quant Dashboard", page_icon="🧪", layout="wide")
st.title("🧪 Litmus Quant Dashboard — TradingView Edition (17 Strategies)")

sym = st.sidebar.text_input("Symbol", "AAPL")
start = st.sidebar.date_input("Start Date", pd.to_datetime("2020-01-01"))
end = st.sidebar.date_input("End Date", pd.to_datetime("2024-01-01"))
cash = st.sidebar.number_input("Starting Cash ($)", min_value=float(1000), max_value=float(1e9), value=float(10000))
sel = st.sidebar.selectbox("Select Strategy", list(STRATS.keys()))
run = st.sidebar.button("🚀 Run Backtest")

if run:
    data = yf.download(sym, start=start, end=end, progress=False)
    data = clean_data(data)
    data["volume"] = data["volume"].astype(float)
    r, ev, sigs, wr, dd = run_bt(data, STRATS[sel], cash)
    st.success(f"✅ {sel}: Return {r}%, Final ${ev:,.2f}, Win Rate {wr}%, Drawdown {dd}%")

    # --- Full TradingView-Style Chart ---
    fig = make_subplots(rows=1, cols=1, shared_xaxes=True)
    fig.add_trace(go.Candlestick(
        x=data.index,
        open=data["open"],
        high=data["high"],
        low=data["low"],
        close=data["close"],
        name="Price",
        increasing_line_color="#00ff99",
        decreasing_line_color="#ff4d4d",
        increasing_fillcolor="rgba(0,255,153,0.35)",
        decreasing_fillcolor="rgba(255,77,77,0.35)",
        whiskerwidth=0.6
    ))

    if "volume" in data.columns:
        max_vol = data["volume"].max()
        fig.add_trace(go.Bar(
            x=data.index,
            y=data["volume"],
            name="Volume",
            marker_color=[
                f"rgba(0,255,153,{0.2 + 0.5*(v/max_vol)})" if data["close"][i] >= data["open"][i]
                else f"rgba(255,77,77,{0.2 + 0.5*(v/max_vol)})"
                for i, v in enumerate(data["volume"])
            ],
            opacity=0.6,
            yaxis="y2"
        ))

    for dt, act, price in sigs:
        fig.add_trace(go.Scatter(
            x=[dt],
            y=[price],
            mode="markers+text",
            text=[act],
            textposition="top center",
            marker_symbol="triangle-up" if act == "BUY" else "triangle-down",
            marker_color="lime" if act == "BUY" else "red",
            marker_size=14,
            hovertext=f"{act} @ {price:.2f}",
            hoverinfo="text",
            name=act,
            opacity=0.9
        ))

    fig.update_layout(
        template="plotly_dark",
        hovermode="x unified",
        dragmode="zoom",
        height=850,
        plot_bgcolor="rgba(13,13,13,1)",
        paper_bgcolor="rgba(13,13,13,1)",
        xaxis_rangeslider_visible=False,
        font=dict(color="white", family="Inter, Arial, sans-serif"),
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        margin=dict(l=40, r=40, t=60, b=40),
        yaxis=dict(domain=[0.25, 1], showgrid=True, gridcolor="rgba(70,70,70,0.2)", zeroline=False),
        yaxis2=dict(domain=[0, 0.2], showgrid=False),
        shapes=[dict(type="line", xref="x", yref="paper", x0=0, x1=0, y0=0, y1=1,
                     line=dict(color="rgba(255,255,255,0.2)", width=1))]
    )

    st.plotly_chart(
        fig,
        use_container_width=True,
        config={
            "scrollZoom": True,
            "displaylogo": False,
            "modeBarButtonsToAdd": [
                "drawline",
                "drawopenpath",
                "eraseshape",
                "zoom2d",
                "pan2d",
                "resetScale2d"
            ],
        },
    )
else:
    st.info("👈 Select strategy and click 🚀 Run Backtest")


Writing litmus_app.py


In [9]:
%%writefile litmus_app.py
import streamlit as st
import yfinance as yf
import backtrader as bt
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os, datetime, importlib
from pathlib import Path

# ---------- Helpers ----------
def clean_data(data):
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)
    data.columns = [c.lower().strip() for c in data.columns]
    data.dropna(inplace=True)
    return data

def ensure_folder():
    folder = Path("Backtests_History")
    folder.mkdir(exist_ok=True)
    return folder

def save_backtest_results(df, fig):
    folder = ensure_folder()
    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_path = folder / f"results_{ts}.csv"
    png_path = folder / f"chart_{ts}.png"
    df.to_csv(csv_path)
    # Safe image export with Kaleido check
    if importlib.util.find_spec("kaleido") is not None:
        try:
            fig.write_image(png_path)
        except Exception as e:
            print("⚠️ Kaleido error:", e)
    else:
        print("⚠️ Kaleido not installed — skipping PNG export.")
    return csv_path, png_path

# ---------- Base Strategy ----------
class BaseStrategy(bt.Strategy):
    def __init__(self):
        self.signals, self.trades = [], []
    def log_signal(self, action):
        self.signals.append((self.data.datetime.date(0), action, self.data.close[0]))
    def notify_trade(self, trade):
        if trade.isclosed:
            self.trades.append(trade.pnl)

# ---------- Main Strategies ----------
class SmaCross(BaseStrategy):
    params = dict(pfast=10, pslow=30)
    def __init__(self):
        super().__init__()
        sma1, sma2 = bt.ind.SMA(period=self.p.pfast), bt.ind.SMA(period=self.p.pslow)
        self.cross = bt.ind.CrossOver(sma1, sma2)
    def next(self):
        if not self.position and self.cross > 0:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.cross < 0:
            self.sell(); self.log_signal("SELL")

class Rsi(BaseStrategy):
    params = dict(period=14, low=30, high=70)
    def __init__(self):
        super().__init__()
        self.rsi = bt.ind.RSI(period=self.p.period)
    def next(self):
        if not self.position and self.rsi < self.p.low:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.rsi > self.p.high:
            self.sell(); self.log_signal("SELL")

class Bollinger(BaseStrategy):
    params = dict(period=20, dev=2)
    def __init__(self):
        super().__init__()
        self.bb = bt.ind.BollingerBands(period=self.p.period, devfactor=self.p.dev)
    def next(self):
        if not self.position and self.data.close < self.bb.bot:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.data.close > self.bb.top:
            self.sell(); self.log_signal("SELL")

class Momentum(BaseStrategy):
    params = dict(period=20)
    def next(self):
        if len(self.data) <= self.p.period:
            return
        if not self.position and self.data.close[0] > self.data.close[-self.p.period]:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.data.close[0] < self.data.close[-self.p.period]:
            self.sell(); self.log_signal("SELL")

class MACD(BaseStrategy):
    params = dict(fast=12, slow=26, sig=9)
    def __init__(self):
        super().__init__()
        m = bt.ind.MACD(period_me1=self.p.fast, period_me2=self.p.slow, period_signal=self.p.sig)
        self.cross = bt.ind.CrossOver(m.macd, m.signal)
    def next(self):
        if not self.position and self.cross > 0:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.cross < 0:
            self.sell(); self.log_signal("SELL")

class BuyHold(BaseStrategy):
    def next(self):
        if not self.position:
            self.buy(); self.log_signal("BUY")

# Placeholder classes for completeness
class EMA(BaseStrategy): pass
class DualSMA(BaseStrategy): pass
class Donchian(BaseStrategy): pass
class Stoch(BaseStrategy): pass
class ADX(BaseStrategy): pass
class VWAP(BaseStrategy): pass
class ATRStop(BaseStrategy): pass
class Seasonal(BaseStrategy): pass
class RiskParity(BaseStrategy): pass
class Factor(BaseStrategy): pass
class Breakout(BaseStrategy): pass

STRATS = {
    "SMA Crossover": SmaCross, "RSI": Rsi, "Bollinger": Bollinger,
    "Momentum": Momentum, "MACD": MACD, "Buy & Hold": BuyHold,
    "EMA": EMA, "Dual SMA": DualSMA, "Donchian": Donchian, "Stoch": Stoch,
    "ADX": ADX, "VWAP": VWAP, "ATR Stop": ATRStop, "Seasonal": Seasonal,
    "Risk Parity": RiskParity, "Factor": Factor, "Breakout": Breakout
}

# ---------- Backtest Engine ----------
def run_bt(data, cls, cash=10000, **kwargs):
    data = clean_data(data.copy())
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(cash)
    feed = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(feed)
    cerebro.addstrategy(cls, **kwargs)
    results = cerebro.run()
    strat = results[0] if results else None
    signals = getattr(strat, "signals", [])
    trades = getattr(strat, "trades", [])
    start = cash
    end = cerebro.broker.getvalue()
    ret = round((end / start - 1) * 100, 2)
    win = round(100 * len([t for t in trades if t > 0]) / len(trades), 2) if trades else 0
    dd = round(abs(min(trades)) / cash * 100, 2) if trades else 0
    return ret, end, signals, win, dd

# ---------- Streamlit App ----------
st.set_page_config(page_title="Litmus Quant Dashboard", page_icon="🧪", layout="wide")
st.title("🧪 Litmus Strategy Lab — Tune, Test & Compare")

mode = st.sidebar.radio("Select Mode", ["🚀 Run Backtest", "📚 Backtest Dashboard", "🧪 Strategy Lab"])

# ===========================================================
# 🚀 MODE 1: Run Backtest
# ===========================================================
if mode == "🚀 Run Backtest":
    st.sidebar.header("Run Settings")
    sym = st.sidebar.text_input("Symbol", "AAPL")
    start = st.sidebar.date_input("Start Date", pd.to_datetime("2020-01-01"))
    end = st.sidebar.date_input("End Date", pd.to_datetime("2024-01-01"))
    cash = st.sidebar.number_input("Starting Cash ($)", 0.9, 1e12, 10000.0)
    selected = st.sidebar.multiselect("Select Strategies", list(STRATS.keys()), default=["SMA Crossover","RSI","Bollinger"])
    run = st.sidebar.button("Run Backtest")

    if run:
        data = yf.download(sym, start=start, end=end, progress=False)
        data = clean_data(data)
        data["volume"] = data["volume"].astype(float)
        results = []
        for sn in selected:
            r, ev, sigs, wr, dd = run_bt(data, STRATS[sn], cash)
            results.append({"Strategy": sn, "Return %": r, "Final Value": ev, "Win Rate %": wr, "Max DD %": dd})
        df = pd.DataFrame(results).set_index("Strategy")
        st.dataframe(df)

        # --- Enhanced Chart ---
        fig = make_subplots(rows=1, cols=1)
        fig.add_trace(go.Candlestick(
            x=data.index, open=data["open"], high=data["high"], low=data["low"], close=data["close"],
            name="Price", increasing_line_color="#00ff99", decreasing_line_color="#ff4d4d",
            increasing_fillcolor="rgba(0,255,153,0.4)", decreasing_fillcolor="rgba(255,77,77,0.4)",
            whiskerwidth=0.6  # longer wicks
        ))

        fig.update_layout(
            template="plotly_dark", hovermode="x unified", dragmode="zoom",
            plot_bgcolor="rgba(13,13,13,1)", paper_bgcolor="rgba(13,13,13,1)",
            xaxis_rangeslider_visible=False, height=850,
            font=dict(color="white", family="Arial"),
            xaxis=dict(showgrid=False), yaxis=dict(showgrid=True, gridcolor="rgba(70,70,70,0.2)")
        )

        st.plotly_chart(fig, use_container_width=True, config={"scrollZoom": True, "displaylogo": False})

# ===========================================================
# 📚 MODE 2: Backtest Dashboard
# ===========================================================
elif mode == "📚 Backtest Dashboard":
    folder = ensure_folder()
    files = sorted(folder.glob("results_*.csv"), reverse=True)
    if not files:
        st.info("No saved backtests yet.")
    else:
        choice = st.sidebar.selectbox("Choose a saved backtest", [f.name for f in files])
        df = pd.read_csv(folder / choice, index_col=0)
        st.dataframe(df)
        chart_path = folder / choice.replace("results_", "chart_").replace(".csv", ".png")
        if chart_path.exists():
            st.image(str(chart_path), use_container_width=True)

# ===========================================================
# 🧪 MODE 3: Strategy Lab
# ===========================================================
else:
    st.sidebar.header("Strategy Lab Settings")
    sym = st.sidebar.text_input("Symbol", "AAPL")
    start = st.sidebar.date_input("Start Date", pd.to_datetime("2020-01-01"))
    end = st.sidebar.date_input("End Date", pd.to_datetime("2024-01-01"))
    cash = st.sidebar.number_input("Starting Cash ($)", 0.9, 1e12, 10000.0)
    strat_choice = st.sidebar.selectbox("Choose Strategy", list(STRATS.keys()))

    # Strategy-specific parameter sliders
    kwargs = {}
    if strat_choice == "SMA Crossover":
        kwargs["pfast"] = st.sidebar.slider("Fast SMA", 5, 50, 10)
        kwargs["pslow"] = st.sidebar.slider("Slow SMA", 20, 200, 30)
    elif strat_choice == "RSI":
        kwargs["period"] = st.sidebar.slider("RSI Period", 5, 30, 14)
        kwargs["low"] = st.sidebar.slider("Buy Threshold", 10, 40, 30)
        kwargs["high"] = st.sidebar.slider("Sell Threshold", 60, 90, 70)
    elif strat_choice == "Bollinger":
        kwargs["period"] = st.sidebar.slider("Period", 10, 40, 20)
        kwargs["dev"] = st.sidebar.slider("Deviation", 1.0, 3.0, 2.0)
    elif strat_choice == "Momentum":
        kwargs["period"] = st.sidebar.slider("Lookback Period", 5, 60, 20)
    elif strat_choice == "MACD":
        kwargs["fast"] = st.sidebar.slider("Fast EMA", 5, 20, 12)
        kwargs["slow"] = st.sidebar.slider("Slow EMA", 20, 40, 26)
        kwargs["sig"] = st.sidebar.slider("Signal", 5, 20, 9)
    else:
        st.sidebar.info("This strategy has no adjustable parameters.")

    run = st.sidebar.button("Run Strategy Lab")

    if run:
        data = yf.download(sym, start=start, end=end, progress=False)
        data = clean_data(data)
        data["volume"] = data["volume"].astype(float)
        r, ev, sigs, wr, dd = run_bt(data, STRATS[strat_choice], cash, **kwargs)
        st.success(f"✅ {strat_choice} — Return {r}%, Final ${ev:,.2f}, Win Rate {wr}%")

        # --- Enhanced Chart ---
        fig = make_subplots(rows=1, cols=1)
        fig.add_trace(go.Candlestick(
            x=data.index, open=data["open"], high=data["high"], low=data["low"], close=data["close"],
            name="Price", increasing_line_color="#00ff99", decreasing_line_color="#ff4d4d",
            increasing_fillcolor="rgba(0,255,153,0.4)", decreasing_fillcolor="rgba(255,77,77,0.4)",
            whiskerwidth=0.6
        ))
        for dt, act, price in sigs:
            fig.add_trace(go.Scatter(x=[dt], y=[price], mode="markers",
                                     marker_symbol="triangle-up" if act=="BUY" else "triangle-down",
                                     marker_color="lime" if act=="BUY" else "red",
                                     marker_size=12, name=act,
                                     hovertext=f"{act}@{price:.2f}"))

        fig.update_layout(
            template="plotly_dark", hovermode="x unified", dragmode="zoom",
            plot_bgcolor="rgba(13,13,13,1)", paper_bgcolor="rgba(13,13,13,1)",
            xaxis_rangeslider_visible=False, height=850,
            font=dict(color="white", family="Arial"),
            xaxis=dict(showgrid=False), yaxis=dict(showgrid=True, gridcolor="rgba(70,70,70,0.2)")
        )
        st.plotly_chart(fig, use_container_width=True, config={"scrollZoom": True, "displaylogo": False})


Overwriting litmus_app.py


In [13]:
# --- PHASE 17: Persistent Settings (Add-on Layer for Litmus App) ---

import streamlit as st
import pandas as pd

# Load persistent state for Litmus app
if "symbol" not in st.session_state:
    st.session_state["symbol"] = "AAPL"
if "strategy" not in st.session_state:
    st.session_state["strategy"] = "SMA Crossover"
if "cash" not in st.session_state:
    st.session_state["cash"] = 10000.0
if "dates" not in st.session_state:
    st.session_state["dates"] = (
        pd.to_datetime("2020-01-01"),
        pd.to_datetime("2024-01-01"),
    )

# Create sticky sidebar controls
st.sidebar.header("📌 Persistent User Settings")

sym = st.sidebar.text_input("Symbol", st.session_state["symbol"])
start = st.sidebar.date_input("Start Date", st.session_state["dates"][0])
end = st.sidebar.date_input("End Date", st.session_state["dates"][1])
cash = st.sidebar.number_input("Starting Cash ($)", 0.9, 1e12, st.session_state["cash"])
strategy = st.sidebar.selectbox(
    "Default Strategy",
    list(st.session_state.get("STRATS", [])) if "STRATS" in st.session_state else [],
    index=0 if "strategy" not in st.session_state else 0,
)

# Update state whenever user changes
st.session_state["symbol"] = sym
st.session_state["dates"] = (start, end)
st.session_state["cash"] = cash
st.session_state["strategy"] = strategy

st.sidebar.success("✅ Preferences saved! These will persist between app reloads.")


2025-11-14 13:05:17.266 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-14 13:05:17.268 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-14 13:05:17.270 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-14 13:05:17.272 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-14 13:05:17.275 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-14 13:05:17.276 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-14 13:05:17.278 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-14 13:05:17.279 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

DeltaGenerator(_root_container=1, _parent=DeltaGenerator())

In [14]:
%%writefile litmus_app.py
import streamlit as st
import yfinance as yf
import backtrader as bt
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os, datetime, importlib
from pathlib import Path

# ---------- Helpers ----------
def clean_data(data):
    if isinstance(data.columns, pd.MultiIndex):
        data.columns = data.columns.get_level_values(0)
    data.columns = [c.lower().strip() for c in data.columns]
    data.dropna(inplace=True)
    return data

def ensure_folder():
    folder = Path("Backtests_History")
    folder.mkdir(exist_ok=True)
    return folder

def save_backtest_results(df, fig):
    folder = ensure_folder()
    ts = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    csv_path = folder / f"results_{ts}.csv"
    png_path = folder / f"chart_{ts}.png"
    df.to_csv(csv_path)
    if importlib.util.find_spec("kaleido") is not None:
        try:
            fig.write_image(png_path)
        except Exception as e:
            print("⚠️ Kaleido error:", e)
    else:
        print("⚠️ Kaleido not installed — skipping PNG export.")
    return csv_path, png_path

# ---------- Base Strategy ----------
class BaseStrategy(bt.Strategy):
    def __init__(self):
        self.signals, self.trades = [], []
    def log_signal(self, action):
        self.signals.append((self.data.datetime.date(0), action, self.data.close[0]))
    def notify_trade(self, trade):
        if trade.isclosed:
            self.trades.append(trade.pnl)

# ---------- Main Strategies ----------
class SmaCross(BaseStrategy):
    params = dict(pfast=10, pslow=30)
    def __init__(self):
        super().__init__()
        sma1, sma2 = bt.ind.SMA(period=self.p.pfast), bt.ind.SMA(period=self.p.pslow)
        self.cross = bt.ind.CrossOver(sma1, sma2)
    def next(self):
        if not self.position and self.cross > 0:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.cross < 0:
            self.sell(); self.log_signal("SELL")

class Rsi(BaseStrategy):
    params = dict(period=14, low=30, high=70)
    def __init__(self):
        super().__init__()
        self.rsi = bt.ind.RSI(period=self.p.period)
    def next(self):
        if not self.position and self.rsi < self.p.low:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.rsi > self.p.high:
            self.sell(); self.log_signal("SELL")

class Bollinger(BaseStrategy):
    params = dict(period=20, dev=2)
    def __init__(self):
        super().__init__()
        self.bb = bt.ind.BollingerBands(period=self.p.period, devfactor=self.p.dev)
    def next(self):
        if not self.position and self.data.close < self.bb.bot:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.data.close > self.bb.top:
            self.sell(); self.log_signal("SELL")

class Momentum(BaseStrategy):
    params = dict(period=20)
    def next(self):
        if len(self.data) <= self.p.period:
            return
        if not self.position and self.data.close[0] > self.data.close[-self.p.period]:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.data.close[0] < self.data.close[-self.p.period]:
            self.sell(); self.log_signal("SELL")

class MACD(BaseStrategy):
    params = dict(fast=12, slow=26, sig=9)
    def __init__(self):
        super().__init__()
        m = bt.ind.MACD(period_me1=self.p.fast, period_me2=self.p.slow, period_signal=self.p.sig)
        self.cross = bt.ind.CrossOver(m.macd, m.signal)
    def next(self):
        if not self.position and self.cross > 0:
            self.buy(); self.log_signal("BUY")
        elif self.position and self.cross < 0:
            self.sell(); self.log_signal("SELL")

class BuyHold(BaseStrategy):
    def next(self):
        if not self.position:
            self.buy(); self.log_signal("BUY")

# Placeholder classes
class EMA(BaseStrategy): pass
class DualSMA(BaseStrategy): pass
class Donchian(BaseStrategy): pass
class Stoch(BaseStrategy): pass
class ADX(BaseStrategy): pass
class VWAP(BaseStrategy): pass
class ATRStop(BaseStrategy): pass
class Seasonal(BaseStrategy): pass
class RiskParity(BaseStrategy): pass
class Factor(BaseStrategy): pass
class Breakout(BaseStrategy): pass

STRATS = {
    "SMA Crossover": SmaCross, "RSI": Rsi, "Bollinger": Bollinger,
    "Momentum": Momentum, "MACD": MACD, "Buy & Hold": BuyHold,
    "EMA": EMA, "Dual SMA": DualSMA, "Donchian": Donchian, "Stoch": Stoch,
    "ADX": ADX, "VWAP": VWAP, "ATR Stop": ATRStop, "Seasonal": Seasonal,
    "Risk Parity": RiskParity, "Factor": Factor, "Breakout": Breakout
}

# ---------- Backtest Engine ----------
def run_bt(data, cls, cash=10000, **kwargs):
    data = clean_data(data.copy())
    cerebro = bt.Cerebro()
    cerebro.broker.setcash(cash)
    feed = bt.feeds.PandasData(dataname=data)
    cerebro.adddata(feed)
    cerebro.addstrategy(cls, **kwargs)
    results = cerebro.run()
    strat = results[0] if results else None
    signals = getattr(strat, "signals", [])
    trades = getattr(strat, "trades", [])
    start = cash
    end = cerebro.broker.getvalue()
    ret = round((end / start - 1) * 100, 2)
    win = round(100 * len([t for t in trades if t > 0]) / len(trades), 2) if trades else 0
    dd = round(abs(min(trades)) / cash * 100, 2) if trades else 0
    return ret, end, signals, win, dd

# ---------- Streamlit App ----------
st.set_page_config(page_title="Litmus Quant Dashboard", page_icon="🧪", layout="wide")
st.title("🧪 Litmus Strategy Lab — Tune, Test & Compare")

mode = st.sidebar.radio(
    "Select Mode",
    ["🚀 Run Backtest", "📚 Backtest Dashboard", "🧪 Strategy Lab", "🌍 Multi-Asset Comparison", "📈 Advanced Analytics"]
)

# ===========================================================
# 🚀 MODE 1: Run Backtest
# ===========================================================
if mode == "🚀 Run Backtest":
    st.sidebar.header("Run Settings")
    sym = st.sidebar.text_input("Symbol", "AAPL")
    start = st.sidebar.date_input("Start Date", pd.to_datetime("2020-01-01"))
    end = st.sidebar.date_input("End Date", pd.to_datetime("2024-01-01"))
    cash = st.sidebar.number_input("Starting Cash ($)", 0.9, 1e12, 10000.0)
    selected = st.sidebar.multiselect("Select Strategies", list(STRATS.keys()), default=["SMA Crossover","RSI","Bollinger"])
    run = st.sidebar.button("Run Backtest")

    if run:
        data = yf.download(sym, start=start, end=end, progress=False)
        data = clean_data(data)
        data["volume"] = data["volume"].astype(float)
        results = []
        for sn in selected:
            r, ev, sigs, wr, dd = run_bt(data, STRATS[sn], cash)
            results.append({"Strategy": sn, "Return %": r, "Final Value": ev, "Win Rate %": wr, "Max DD %": dd})
        df = pd.DataFrame(results).set_index("Strategy")
        st.dataframe(df)

        # Enhanced Chart
        fig = make_subplots(rows=1, cols=1)
        fig.add_trace(go.Candlestick(
            x=data.index, open=data["open"], high=data["high"], low=data["low"], close=data["close"],
            name="Price", increasing_line_color="#00ff99", decreasing_line_color="#ff4d4d",
            increasing_fillcolor="rgba(0,255,153,0.4)", decreasing_fillcolor="rgba(255,77,77,0.4)",
            whiskerwidth=0.6))
        fig.update_layout(template="plotly_dark", hovermode="x unified", dragmode="zoom",
                          plot_bgcolor="rgba(13,13,13,1)", paper_bgcolor="rgba(13,13,13,1)",
                          xaxis_rangeslider_visible=False, height=850,
                          font=dict(color="white", family="Arial"))
        st.plotly_chart(fig, use_container_width=True, config={"scrollZoom": True, "displaylogo": False})

# ===========================================================
# 📚 MODE 2: Backtest Dashboard
# ===========================================================
elif mode == "📚 Backtest Dashboard":
    folder = ensure_folder()
    files = sorted(folder.glob("results_*.csv"), reverse=True)
    if not files:
        st.info("No saved backtests yet.")
    else:
        choice = st.sidebar.selectbox("Choose a saved backtest", [f.name for f in files])
        df = pd.read_csv(folder / choice, index_col=0)
        st.dataframe(df)
        chart_path = folder / choice.replace("results_", "chart_").replace(".csv", ".png")
        if chart_path.exists():
            st.image(str(chart_path), use_container_width=True)

# ===========================================================
# 🧪 MODE 3: Strategy Lab
# ===========================================================
elif mode == "🧪 Strategy Lab":
    st.sidebar.header("Strategy Lab Settings")
    sym = st.sidebar.text_input("Symbol", "AAPL")
    start = st.sidebar.date_input("Start Date", pd.to_datetime("2020-01-01"))
    end = st.sidebar.date_input("End Date", pd.to_datetime("2024-01-01"))
    cash = st.sidebar.number_input("Starting Cash ($)", 0.9, 1e12, 10000.0)
    strat_choice = st.sidebar.selectbox("Choose Strategy", list(STRATS.keys()))

    kwargs = {}
    if strat_choice == "SMA Crossover":
        kwargs["pfast"] = st.sidebar.slider("Fast SMA", 5, 50, 10)
        kwargs["pslow"] = st.sidebar.slider("Slow SMA", 20, 200, 30)
    elif strat_choice == "RSI":
        kwargs["period"] = st.sidebar.slider("RSI Period", 5, 30, 14)
        kwargs["low"] = st.sidebar.slider("Buy Threshold", 10, 40, 30)
        kwargs["high"] = st.sidebar.slider("Sell Threshold", 60, 90, 70)

    run = st.sidebar.button("Run Strategy Lab")

    if run:
        data = yf.download(sym, start=start, end=end, progress=False)
        data = clean_data(data)
        data["volume"] = data["volume"].astype(float)
        r, ev, sigs, wr, dd = run_bt(data, STRATS[strat_choice], cash, **kwargs)
        st.success(f"✅ {strat_choice} — Return {r}%, Final ${ev:,.2f}, Win Rate {wr}%")

        fig = make_subplots(rows=1, cols=1)
        fig.add_trace(go.Candlestick(x=data.index, open=data["open"], high=data["high"],
                                     low=data["low"], close=data["close"], name="Price",
                                     increasing_line_color="#00ff99", decreasing_line_color="#ff4d4d",
                                     increasing_fillcolor="rgba(0,255,153,0.4)", decreasing_fillcolor="rgba(255,77,77,0.4)",
                                     whiskerwidth=0.6))
        for dt, act, price in sigs:
            fig.add_trace(go.Scatter(x=[dt], y=[price], mode="markers",
                                     marker_symbol="triangle-up" if act=="BUY" else "triangle-down",
                                     marker_color="lime" if act=="BUY" else "red",
                                     marker_size=12, name=act, hovertext=f"{act}@{price:.2f}"))
        fig.update_layout(template="plotly_dark", hovermode="x unified", dragmode="zoom",
                          plot_bgcolor="rgba(13,13,13,1)", paper_bgcolor="rgba(13,13,13,1)",
                          xaxis_rangeslider_visible=False, height=850,
                          font=dict(color="white", family="Arial"))
        st.plotly_chart(fig, use_container_width=True, config={"scrollZoom": True, "displaylogo": False})

# ===========================================================
# 🌍 MODE 4: Multi-Asset Comparison
# ===========================================================
# ===========================================================
# 🌍 MODE 4: Multi-Asset Comparison
# ===========================================================
elif mode == "🌍 Multi-Asset Comparison":
    st.sidebar.header("Multi-Asset Comparison Settings")
    symbols_input = st.sidebar.text_input("Enter tickers (comma-separated)", "AAPL, MSFT, TSLA, NVDA")
    strategy_choice = st.sidebar.selectbox("Select Strategy", list(STRATS.keys()))
    start = st.sidebar.date_input("Start Date", pd.to_datetime("2020-01-01"))
    end = st.sidebar.date_input("End Date", pd.to_datetime("2024-01-01"))
    cash = st.sidebar.number_input("Starting Cash ($)", 0.9, 1e12, 10000.0)
    run_multi = st.sidebar.button("🚀 Run Multi-Asset Backtest")

    if run_multi:
        symbols = [s.strip().upper() for s in symbols_input.split(",") if s.strip()]
        results = []
        perf_chart = make_subplots(rows=1, cols=1, shared_xaxes=True)
        for sym in symbols:
            try:
                data = yf.download(sym, start=start, end=end, progress=False)
                data = clean_data(data)
                r, ev, sigs, wr, dd = run_bt(data, STRATS[strategy_choice], cash)
                results.append({"Symbol": sym, "Return %": r, "Final Value": ev, "Win Rate %": wr, "Max DD %": dd})
                perf_chart.add_trace(go.Scatter(x=data.index, y=data["close"], mode="lines", name=sym))
            except Exception as e:
                st.warning(f"⚠️ {sym}: {e}")

        if results:
            df = pd.DataFrame(results).set_index("Symbol").sort_values("Return %", ascending=False)
            st.dataframe(df)
            perf_chart.update_layout(template="plotly_dark", title=f"Price Comparison ({strategy_choice})",
                                     height=750, hovermode="x unified",
                                     plot_bgcolor="rgba(13,13,13,1)", paper_bgcolor="rgba(13,13,13,1)",
                                     font=dict(color="white"))
            st.plotly_chart(perf_chart, use_container_width=True, config={"scrollZoom": True, "displaylogo": False})


# ===========================================================
# ===========================================================
# ===========================================================
# 📈 MODE 5: Advanced Analytics (Sharpe, Volatility, Profit Factor, Drawdowns)
# ===========================================================
elif mode == "📈 Advanced Analytics":
    st.sidebar.header("Advanced Analytics Settings")
    symbols_input = st.sidebar.text_input("Enter tickers (comma-separated)", "AAPL, MSFT, TSLA, NVDA")
    strategy_choice = st.sidebar.selectbox("Select Strategy", list(STRATS.keys()))
    start = st.sidebar.date_input("Start Date", pd.to_datetime("2020-01-01"))
    end = st.sidebar.date_input("End Date", pd.to_datetime("2024-01-01"))
    cash = st.sidebar.number_input("Starting Cash ($)", 0.01, 1e12, 10000.0)
    run_adv = st.sidebar.button("🚀 Run Advanced Analytics")

    if run_adv:
        symbols = [s.strip().upper() for s in symbols_input.split(",") if s.strip()]
        results = []
        chart = make_subplots(rows=1, cols=1, shared_xaxes=True)
        last_data = None  # store last dataset for charts

        for sym in symbols:
            try:
                # --- Fetch & prepare data ---
                data = yf.download(sym, start=start, end=end, progress=False)
                data = clean_data(data)
                data["returns"] = data["close"].pct_change()
                mean_ret = data["returns"].mean()
                std_ret = data["returns"].std()
                sharpe = (mean_ret / std_ret) * np.sqrt(252) if std_ret > 0 else 0
                vol = std_ret * np.sqrt(252) * 100

                # --- Run backtest ---
                r, ev, sigs, wr, dd = run_bt(data, STRATS[strategy_choice], cash)

                # --- Equity & Profit/Loss ---
                data["equity_curve"] = (1 + data["returns"]).cumprod() * cash
                data["profit_loss"] = data["equity_curve"] - cash
                rolling_max = data["equity_curve"].cummax()
                drawdown = (data["equity_curve"] - rolling_max) / rolling_max
                max_dd = round(drawdown.min() * 100, 2)
                cum_pl = data["profit_loss"].iloc[-1]
                last_data = data.copy()

                # --- Trade Metrics ---
                trades = getattr(run_bt(data, STRATS[strategy_choice], cash), "trades", [])
                if trades:
                    wins = [t for t in trades if t > 0]
                    losses = [t for t in trades if t < 0]
                    success_rate = (len(wins) / len(trades)) * 100
                    profit_factor = (sum(wins) / abs(sum(losses))) if losses else float('inf')
                else:
                    success_rate = 0
                    profit_factor = 0

                # --- Append Results ---
                results.append({
                    "Symbol": sym,
                    "Return %": r,
                    "Final Value": ev,
                    "Sharpe Ratio": round(sharpe, 2),
                    "Volatility %": round(vol, 2),
                    "Win Rate %": wr,
                    "Success Rate %": round(success_rate, 2),
                    "Profit Factor": round(profit_factor, 2),
                    "Cumulative P/L ($)": round(cum_pl, 2),
                    "Max Drawdown %": max_dd
                })

                # --- Add Equity to Chart ---
                chart.add_trace(go.Scatter(
                    x=data.index, y=data["equity_curve"],
                    mode="lines", name=f"{sym}",
                    line=dict(width=2.5),
                    hovertemplate=f"{sym}<br>Equity: %{{y:.2f}}<extra></extra>"
                ))

            except Exception as e:
                st.warning(f"⚠️ {sym}: {e}")

        # --- Display All Metrics ---
        if results:
            df = pd.DataFrame(results).set_index("Symbol").sort_values("Sharpe Ratio", ascending=False)

            # --- Summary KPI Cards ---
            best_symbol = df["Return %"].idxmax()
            best_return = df["Return %"].max()
            worst_dd_symbol = df["Max Drawdown %"].idxmin()
            worst_dd = df["Max Drawdown %"].min()
            best_pf_symbol = df["Profit Factor"].idxmax()
            best_pf = df["Profit Factor"].max()
            avg_return = df["Return %"].mean()

            kpi1, kpi2, kpi3, kpi4 = st.columns(4)
            kpi1.metric("🥇 Best Symbol", best_symbol, f"{best_return:.2f}% Return")
            kpi2.metric("📉 Worst Drawdown", worst_dd_symbol, f"{worst_dd:.2f}% DD")
            kpi3.metric("💰 Highest Profit Factor", best_pf_symbol, f"{best_pf:.2f}")
            kpi4.metric("📈 Avg Return (All)", "", f"{avg_return:.2f}%")

            # --- Metrics Table ---
            st.subheader("📊 Advanced Performance Table")
            st.dataframe(df.style.format({
                "Return %": "{:.2f}",
                "Volatility %": "{:.2f}",
                "Sharpe Ratio": "{:.2f}",
                "Win Rate %": "{:.2f}",
                "Success Rate %": "{:.2f}",
                "Profit Factor": "{:.2f}",
                "Cumulative P/L ($)": "{:.2f}",
                "Max Drawdown %": "{:.2f}"
            }))

            # --- Portfolio Summary ---
            total_initial = cash * len(df)
            total_final = df["Final Value"].sum()
            portfolio_return = ((total_final / total_initial) - 1) * 100

            combined_equity = pd.DataFrame()
            for sym in symbols:
                data = yf.download(sym, start=start, end=end, progress=False)
                data = clean_data(data)
                data["returns"] = data["close"].pct_change()
                data["equity"] = (1 + data["returns"]).cumprod() * cash
                combined_equity[sym] = data["equity"]

            combined_equity["portfolio"] = combined_equity.mean(axis=1)
            combined_equity["returns"] = combined_equity["portfolio"].pct_change()
            sharpe_portfolio = (combined_equity["returns"].mean() / combined_equity["returns"].std()) * np.sqrt(252)
            rolling_max_portfolio = combined_equity["portfolio"].cummax()
            drawdown_portfolio = (combined_equity["portfolio"] - rolling_max_portfolio) / rolling_max_portfolio
            max_dd_portfolio = drawdown_portfolio.min() * 100

            st.markdown("### 📘 Portfolio Summary")
            colA, colB, colC, colD, colE = st.columns(5)
            colA.metric("💵 Total Initial", f"${total_initial:,.2f}")
            colB.metric("💰 Total Final Value", f"${total_final:,.2f}")
            colC.metric("📈 Portfolio Return %", f"{portfolio_return:.2f}%")
            colD.metric("⚙️ Portfolio Sharpe", f"{sharpe_portfolio:.2f}")
            colE.metric("📉 Max Drawdown %", f"{max_dd_portfolio:.2f}")

            # --- Safe P/L & Drawdown Charts ---
            if last_data is not None and "profit_loss" in last_data.columns:
                plot_df = last_data.copy()
                rolling_max = plot_df["equity_curve"].cummax()
                plot_df["drawdown"] = (plot_df["equity_curve"] - rolling_max) / rolling_max * 100

                profit_fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                                           subplot_titles=("Cumulative Profit/Loss", "Drawdown (%)"))

                profit_fig.add_trace(go.Scatter(
                    x=plot_df.index, y=plot_df["profit_loss"],
                    mode="lines", name="Cumulative P/L",
                    line=dict(width=2, color="lime"),
                    hovertemplate="%{x|%Y-%m-%d}<br>P/L: %{y:.2f}<extra></extra>"
                ), row=1, col=1)

                profit_fig.add_trace(go.Scatter(
                    x=plot_df.index, y=plot_df["drawdown"],
                    mode="lines", name="Drawdown (%)",
                    line=dict(width=2, color="red"),
                    hovertemplate="%{x|%Y-%m-%d}<br>Drawdown: %{y:.2f}%<extra></extra>"
                ), row=2, col=1)

                profit_fig.update_layout(
                    template="plotly_dark",
                    height=700,
                    plot_bgcolor="rgba(13,13,13,1)",
                    paper_bgcolor="rgba(13,13,13,1)",
                    font=dict(color="white"),
                    hovermode="x unified",
                    title="📉 Cumulative Profit/Loss & Drawdown"
                )

                st.plotly_chart(chart, use_container_width=True, config={"scrollZoom": True, "displaylogo": False})
                st.plotly_chart(profit_fig, use_container_width=True, config={"scrollZoom": True, "displaylogo": False})
            else:
                st.warning("⚠️ No profit/loss data available for visualization.")

           
           

Overwriting litmus_app.py
